# Metaprogramación

- Todo en python es un "objeto", esto incluye las clases
- Una clase es un objeto que se puede utilizar para crear otros objetos. 
- Una metaclase esá por encima de la clase. Es un objeto que se usa para crear clases.

In [2]:
def get_curso_alfonso():
    class CursoAlfonso:
        pass
    
    return CursoAlfonso

print(get_curso_alfonso())

<class '__main__.get_curso_alfonso.<locals>.CursoAlfonso'>


In [4]:
# método type
def get_curso_alfonso():
    class CursoAlfonso:
        pass
    
    return CursoAlfonso

print(type(get_curso_alfonso))
print(type(get_curso_alfonso()))


<class 'function'>
<class 'type'>


In [9]:
class Blog:
    def __init__(self, blog_name):
        self._blog_name = blog_name
        
    def get_blog_name(self):
        return f'Blog name is {self._blog_name}'
    
blog = Blog("python")
print(blog.get_blog_name())

Blog name is python


In [7]:
Blog.__dict__

mappingproxy({'__module__': '__main__',
              '__init__': <function __main__.Blog.__init__(self, blog_name)>,
              'get_blog_name': <function __main__.Blog.get_blog_name(self)>,
              '__dict__': <attribute '__dict__' of 'Blog' objects>,
              '__weakref__': <attribute '__weakref__' of 'Blog' objects>,
              '__doc__': None})

In [11]:
def blog2_init(self, blog_name):
    self._blog_name = blog_name

Blog2 = type(
    "Blog2", # nombre de la clase
    (), # tupla de herencias
    {
        "__init__": blog2_init,
        "get_blog_name": lambda self: f'Blog name is {self._blog_name}'
    } # diccionario
)

In [12]:
type(Blog2)

type

In [14]:
b = Blog2("otra cosa")

In [15]:
b.get_blog_name()

'Blog name is otra cosa'

In [16]:
Blog2.__dict__

mappingproxy({'__init__': <function __main__.blog2_init(self, blog_name)>,
              'get_blog_name': <function __main__.<lambda>(self)>,
              '__module__': '__main__',
              '__dict__': <attribute '__dict__' of 'Blog2' objects>,
              '__weakref__': <attribute '__weakref__' of 'Blog2' objects>,
              '__doc__': None})

In [17]:
    
ParentClass = type("ParentClass", (), {"parent_function": lambda _ : print("hola desde el padre")})

ChildClass = type ("ChildClass", (ParentClass, ), {
    "variable": 5,
    "child_function": lambda _ : print("hola desde el hijo")
})

In [18]:
c = ChildClass()

In [19]:
c.child_function()

hola desde el hijo


In [20]:
c.parent_function()

hola desde el padre


In [21]:
c.variable

5

In [30]:
# Otro ejemplo de class factory

def manzana():
    """
        devuelve una clase manzana, creada con la keyword class
    """
    class Manzana():
        def __init__(self,  color):
            self._color = color
        
        def get_color(self):
            return self._color
        
    return Manzana

In [31]:
Manzana = manzana()

In [32]:
m = Manzana("roja")

In [33]:
m.get_color()

'roja'

 # Decoradores de clase

In [37]:
def decorator(cls):

    def decorator_init(self):
        print ("en el decorador")
        
    cls.__init__ = decorator_init
    
    return cls

@decorator
class Spam:
    def __init__(self):
        print("init de la clase Spam")
    
    

In [38]:
Spam()

en el decorador


Una metaclase lo que hace es:
- Modifica la clase
- Devuelve la clase

In [41]:
# sintaxis básica de una metaclase

class SpamMetaClass(type):
    def __new__(mcs, name, bases, namespace):
        print ("creando clase")
        return type.__new__(mcs, name, bases, namespace)
    


In [42]:
class Egg(object, metaclass=SpamMetaClass):
    pass

creando clase


In [43]:
Egg()

# Para qué usar metaclases:


django orm
```python
class Person(models.Model):
    name = models.CharField(max_length=30)
    age = models.IntegerField()
```

In [49]:
# una clase abstracta que dice a sus hijos que han de implementar el método get_blog

class SpamMetaClass(type):
    def __new__(mcs, name, bases, dictionary):
        print (dictionary)
        if 'get_blog' in dictionary:
            print ("correcto, estas implementando el método abstracto")
        else:
            raise Exception("No está el método get_blog")
        return type.__new__(mcs, name, bases, dictionary)

In [50]:
class Spam(metaclass = SpamMetaClass):
    pass

{'__module__': '__main__', '__qualname__': 'Spam'}


Exception: No está el método get_blog

In [51]:
class Spam(metaclass = SpamMetaClass):
     def get_blog():
            pass

{'__module__': '__main__', '__qualname__': 'Spam', 'get_blog': <function Spam.get_blog at 0x7f504048a680>}
correcto, estas implementando el método abstracto


Usos posibles:
- comprobar que una clase está definida correctamente (validar)
- levantar errores al importar modulos
- si queremos que cada módulo de nuestro framework tenga una convención de nombre
- generadoress de código
- modificar propiedades al construir la clase

In [54]:
class SpamMetaClass(type):
    def __new__(mcs, name, bases, dictionary):
        print (dictionary)
        if 'get_blog' in dictionary:
            print ("correcto, estas implementando el método abstracto")
        else:
            raise Exception("No está el método get_blog")
            
        new_dict = {}
        for k, v in dictionary.items():
            new_dict[f'alfonso_{k}'] = v

        return type.__new__(mcs, name, bases, new_dict)

In [59]:
class Spam(metaclass = SpamMetaClass):
    variable = 5
    
    def get_blog(self):
            pass

{'__module__': '__main__', '__qualname__': 'Spam', 'variable': 5, 'get_blog': <function Spam.get_blog at 0x7f504048aa70>}
correcto, estas implementando el método abstracto


In [60]:
s = Spam()

In [61]:
s.get_blog()

AttributeError: 'Spam' object has no attribute 'get_blog'

In [62]:
s.alfonso_get_blog()

In [63]:
s.alfonso_variable

5

In [64]:
class Field:
    def __init__(self, name, column_type):
        self.name = name
        self.column_type = column_type
 
    def __str__(self):
        return '<%s:%s>' % (self.__class__.__name__, self.name)
 
 
class StringField(Field):
    def __init__(self, name):
        super(StringField, self).__init__(name, 'varchar(100)')
 
 
class IntegerField(Field):
    def __init__(self, name):
        super(IntegerField, self).__init__(name, 'bigint')
 
 
class ModelMetaclass(type):
    def __new__(mcs, name, bases, attrs):
        if name == 'Model':
            return type.__new__(mcs, name, bases, attrs)
        print("Found Model: %s" % name)
        mapping = dict()
        fields = list()
        # Guarda el atributo en el map
        for k, v in attrs.items():
            if isinstance(v, Field):
                print('Mapeo encontrado : %s ==> %s' % (k, v))
                mapping[k] = v
                fields.append(k)
                 # Eliminar campo en modelo
        for k in mapping.keys():
            attrs.pop(k)
 
        attrs['__fields__'] = list(map(lambda f: '`%s`' % f, fields))
        attrs['__mapping__'] = mapping
        attrs['__table__'] = name
        return type.__new__(mcs, name, bases, attrs)
 
 
class Model(dict, metaclass=ModelMetaclass):
    def __init__(self, **kwargs):
        super(Model, self).__init__(kwargs)
 
    def __getattr__(self, key):
        try:
            return self[key]
        except KeyError:
            raise AttributeError(r"'Model' object has no attribute '%s'" % key)
 
    def __setattr__(self, key, value):
        self[key] = value
 
    def save(self):
        fields = []
        params = []
        args = []
 
        for k, v in self.__mapping__.items():
            print("%s------%s" % (k, v))
            fields.append(v.name)
            params.append('?')
            args.append(getattr(self, k, None))
 
        sql = 'insert into %s (%s) values (%s)' % (self.__table__, ','.join(self.__fields__), ','.join(params))
        print('SQL: %s' % sql)
        print('ARGS: %s' % str(args))
 
 
class User(Model):
         # Defina el mapeo de atributos de clase a columnas:
    id = IntegerField('id')
    name = StringField('username')
    email = StringField('email')
    password = StringField('password')
 
 
u = User(id=12345, name='Michael', email='test@orm.org', password='my-pwd')
u.save()

Found Model: User
Mapeo encontrado : id ==> <IntegerField:id>
Mapeo encontrado : name ==> <StringField:username>
Mapeo encontrado : email ==> <StringField:email>
Mapeo encontrado : password ==> <StringField:password>
id------<IntegerField:id>
name------<StringField:username>
email------<StringField:email>
password------<StringField:password>
SQL: insert into User (`id`,`name`,`email`,`password`) values (?,?,?,?)
ARGS: [12345, 'Michael', 'test@orm.org', 'my-pwd']
